In [1]:
import numpy as np
from model import *
import tensorflow as tf
import setGPU
#strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1"])

imgs = np.load('../dataset/cv/train/x/imgs.npz')['arr_0']
labels = np.load('../dataset/cv/train/y/labels.npy')

setGPU: Setting GPU to: 0


In [2]:
characters = set(char for label in labels for char in label)

In [3]:
# Mapping characters to integers
char_to_num = layers.experimental.preprocessing.StringLookup(
                  vocabulary=sorted(list(characters)), num_oov_indices=0, mask_token=None )

# Mapping integers back to original characters
num_to_char = layers.experimental.preprocessing.StringLookup(
                  vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True )

In [4]:
imgs = np.transpose(imgs, (0, 2, 1, 3))
'''
max_len = max([len(label) for label in labels])
encoded_labels = []
for label in labels:
    if label:
        tmp = np.array([char_to_num(char) for char in label])
        encoded_labels.append(np.pad(tmp, (0, max_len-tmp.shape[0]), constant_values=0))
    else:
        tmp = np.array([char_to_num('#')])
        encoded_labels.append(np.pad(tmp, (0, max_len-tmp.shape[0]), constant_values=0))
'''

encoded_labels = np.load('../dataset/cv/train/y/encoded_labels.npy')

In [5]:
#Si inserisce alla fine di ogni array label la lunghezza reale della parola
t_dist_dim = int(128)          # Questo valore indica quanti step temporali ci sono: nelle ultime feature map ci sono 128/4
                                 # step temporali perché 128 è la larghezza massima tra le immagini in ingresso e 4 è il fattore
                                 # di riduzione dovuto ai MaxPooling (ci sono 2 livelli di MaxPooling che dimezzano le dimensioni)
enc2 = []
for i in range(encoded_labels.shape[0]):
    if len(labels[i]) == 0:
        enc2.append(np.append(encoded_labels[i], [1, t_dist_dim]))
    else:
        enc2.append(np.append(encoded_labels[i], [len(labels[i]), t_dist_dim]))
enc2 = np.array(enc2)

In [6]:
#with strategy.scope():
net = build_and_compile_model_v6(input_shape=(128, 128, 3), len_characters=len(characters), opt="Adam")

In [7]:
net.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 64, 64)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 64, 128)      73856     
_________________________________________________________________
activation_1 (Activation)    (None, 128, 64, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 64, 256)     

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Early_Stopping_Patience = 10 
Min_Delta = 0.0001
# Model Check Point
Check_Point = ModelCheckpoint( 'weights/CRNN_v6.h5',   # Filepath
                               monitor='val_loss',
                               save_best_only=True,
                               verbose=1,
                               mode='auto',
                               save_weights_only=False,
                               save_freq='epoch' )
# Add early stopping
Early_Stopping = EarlyStopping( monitor='val_loss',
                                min_delta=Min_Delta,
                                patience=Early_Stopping_Patience,
                                verbose=1,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True )
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.0002)

In [10]:
batch_size = 32
x_train = imgs
y_train = enc2

In [ ]:
history = net.fit(x_train, y_train, validation_split=0.2, epochs=100, callbacks=[Check_Point, reduce_lr], batch_size=batch_size)

Epoch 1/100
119/906 [==>...........................] - ETA: 2:58 - loss: 44.4136

In [12]:
np.save('my_history_v4.npy',history.history)

In [13]:
history.history

{'loss': [29.55524253845215,
  23.871219635009766,
  22.930871963500977,
  22.60465431213379,
  22.413803100585938,
  22.198286056518555,
  22.000946044921875,
  21.785634994506836,
  21.498363494873047,
  21.11525535583496,
  20.771413803100586,
  20.312236785888672,
  19.845294952392578,
  19.25650405883789,
  18.68903350830078,
  18.20015525817871,
  17.712413787841797,
  17.3137264251709,
  17.127174377441406,
  17.009464263916016,
  16.68659782409668,
  16.430591583251953,
  16.270002365112305,
  16.138019561767578,
  15.820940971374512,
  15.532114028930664,
  15.313992500305176,
  16.10277557373047,
  15.31989574432373,
  15.262053489685059,
  15.380794525146484,
  15.673229217529297,
  15.006155014038086,
  14.855964660644531,
  14.804557800292969,
  14.627645492553711,
  14.880324363708496,
  14.981422424316406,
  15.629707336425781,
  16.130992889404297,
  15.34249210357666,
  14.98897933959961,
  14.976032257080078,
  14.305858612060547,
  14.606742858886719,
  14.5214748382